# Kőnig's theorem

## Minimal vertex cover in a bipartite graph

A *vertex cover* in a graph is a set of vertices that includes at least one endpoint of every edge, and a vertex cover is minimum if no other vertex cover has fewer vertices. A *matching* in a graph is a set of edges no two of which share an endpoint, and a matching is maximum if no other matching has more edges.

It is obvious from the definition that any vertex cover must be with at least the same size as any matching (since for every edge in the matching, at least one vertex is needed in the cover). In particular, the minimum vertex cover set is at least as large as the maximum matching set. Meanwhile, all the vertices in a maximal matching is obviously a vertex cover, so the size of a minimal vertex cover is as most double the size of a maximal matching.

For general graphs, to find a maximal matching is not hard. However, unfortunately, to find a minimal vertex cover is NP-complete. Hence, in this page, we will only discuss the cases of bipartite graphs where, these two problems are equivalent to each other.

## Kőnig's theorem

Kőnig's theorem, proved by Dénes Kőnig (Dénes Kőnig (September 21, 1884 – October 19, 1944) was a Hungarian mathematician of Hungarian Jewish heritage who worked in and wrote the first textbook on the field of graph theory.) in 1931, describes the equivalence between the maximum matching problem and the minimum vertex cover problem in bipartite graphs. It was discovered independently, also in 1931, by Jenő Egerváry in the more general case of weighted graphs.

**Kőnig's theorem.**
In any bipartite graph, the number of edges in a maximum matching equals the number of vertices in a minimum vertex cover.

**Proof 1.** (Using Hall's Marriage Theorem)
We shall show that we can find a maximal matching whose number of edges is equal to the number of vertices in a minimal vertex cover. Given a bipartite graph $G = (X\sqcup Y,E)$ and a minimal vertex cover $C = S\sqcup T$ with $S \subseteq X, T \subseteq Y$. For the induced subgraphs $G_1 = G(S\sqcup (Y\setminus T))$ and $G_2 = G(T\sqcup(X\setminus S))$, we can check the Hall's marriage condition holds on both cases, which will give us a perfect matching $M_1$ in $G_1$ and $M_2$ in $G_2$. $M = M_1 \sqcup M_2$ is a maximal matching we are looking for.
<p style="text-align:right;">&#9632; </p>

The proof above is mathematically easy, but not constructive, which is hard to be used to give any minimal vertex cover.

**Proof 2.** (Constructive proof)
We shall show that we can find a minimal vertex cover whose number of vertices is equal to the number of edges in a maximal matching. Given a bipartite graph $G = (X\sqcup Y,E)$ and a maximal matching $M$ such that $M \bigcap X = S_0$ and $M \bigcap Y = T_0$. Using $M$, we can assign directions for each edge in $G$ by
* $e = \{x,y\}$ goes from $x$ to $y$ if $e \notin M$
* $e = \{x,y\}$ goes from $y$ to $x$ if $e \in M$

Consider the connect component $C_x$ of $x \in X \setminus S_0$ and let $C_x \bigcap X = S_x$ and $C_x \bigcap Y = T_x$, $T_x \subseteq T_0$ by the assumption that $M$ is maximal. Let
$$
    C = \Big(S_0\setminus \big(\bigcup_{x\in X\setminus S_0} S_x\big)\Big) \bigsqcup \Big(\bigcup_{x\in X\setminus S_0} T_x\Big)
$$

Then for any $e = (x,y)\in E$, either $x\in S_0$ or $y \in T_0$.
- If $x \in S_0$, then either $x \in C$, or $y \in C$
- If $x \notin S_0$, then $y\in T_x \subseteq C$

Hence, $C$ is a vertex cover.
<p style="text-align:right;">&#9632; </p>

In [48]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["animation.html"] = "jshtml"
import matplotlib.animation
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import random
import matplotlib.patches as patch
from IPython.display import display,clear_output
from celluloid import Camera

input_text = widgets.Text(
    value='',
    placeholder='Input a 2D list like [[0,],[2,],[1,3,]]',
    description='Input array',
    disabled=False,  
    continuous_update= False
)


output_widget = widgets.Output()

display(input_text)

def HKK(change):
    a = change['new']
    with output_widget:
        
        clear_output()
        
        # check if the input is a 2D list ###
        if not a:
            print('Input is empty, try to input a 2D list')
            return
        try:
            girls = eval(a)
        except:
            print("Illegal inputs")
            return
        try:
            if type(girls) != list:
                print("Input should be like [[0,],[2,],[1,3,]]")
                return
            for girl in girls:
                if type(girl) != list:
                
                    print("Input should be like [[0,],[2,],[1,3,]]")
                    for boy in girl:
                        if type(boy) != int:
                            raise Exception("Illegal inputs")
                    return
        except:
            print("Illegal inputs")
            return
    real_time = 0

        # create a hash map to record boys ##
    boy_set = {}
    for g in range(len(girls)):
        girl = girls[g]
        for boy in girl:
            if boy not in boy_set:
                boy_set[boy]=[]
            boy_set[boy].append(g)
        # prepare the girls and boys nodes ##
    m = max(len(boy_set),len(girls),1)
    Fig = plt.figure(figsize=(4, (m+1)/2+1), layout="constrained")
    subfigs = Fig.subfigures(2, 1, height_ratios=[1,(m+1)/2])
    text_fig,fig = subfigs
    axs = fig.subplots(1, 2)
    hints = text_fig.subplots(2,1)
    for ax in hints:
        ax.set_axis_off()
    title,hint = hints[0],hints[1]
    
    camera = Camera(Fig)
    while True:
        title_text = title.text(0,0,'Ford–Fulkerson Algorithm')
        hint_text = hint.text(0,0,'')
        escape = False
        time = 0
        c = 0
        
        ax = axs[0]
        ax.set_xlim(-1.5,2.5)
        ax.set_ylim(-.5,m-0.5)
        ax.set_aspect('equal')
        ax.set_axis_off()

        for b in boy_set:
            for g in boy_set[b]:
                x0,y0 = -0.8,g + 0.1
                line, = ax.plot(np.array([x0,1.2]), np.array([y0,c+0.1]),color = '#808080',zorder = 0)
            c += 1
            
        # set boxes for girls ######
        for g in range(len(girls)):
            box = patch.FancyBboxPatch(
                (-1, g), 0.4, 0.2,
                boxstyle=patch.BoxStyle("Round", pad=0.2),
                color = '#FFD700',
                zorder = 5
            )
            ax.add_artist(box)
            text = ax.text(
                -0.8, 0.1+g,
                f'Girl {g}', 
                color='w', 
                style = 'italic',
                weight='bold', 
                fontsize=7, 
                ha='center', 
                va='center',
                zorder = 10)
        c = 0
            
        
        # set boxes for boys ######
        for b in boy_set:
            box = patch.FancyBboxPatch(
                (1, c), 0.4, 0.2,
                boxstyle=patch.BoxStyle("Round", pad=0.2),
                color = '#0066CC',
                zorder = 5
            )
            ax.add_artist(box)
            text = ax.text(
                1.2, 0.1+c,
                f'Boy {b}', 
                color='w', 
                style = 'italic',
                weight='bold', 
                fontsize=7, 
                ha='center', 
                va='center',
                zorder = 10
            )
            c += 1
        ax = axs[1]
        ax.set_xlim(-1.5,2.5)
        ax.set_ylim(-.5,m-0.5)
        ax.set_aspect('equal')
        ax.set_axis_off()
        c = 0
        lines = {}
        for b in boy_set:
            for g in boy_set[b]:
                x0,y0 = -0.8,g + 0.1
                line, = ax.plot(np.array([x0,1.2]), np.array([y0,c+0.1]),color = '#808080',zorder = 0)
                lines[(g,b)] = line
            c += 1
            
        girls_box = {}
        for g in range(len(girls)):
            box = patch.FancyBboxPatch(
                (-1, g), 0.4, 0.2,
                boxstyle=patch.BoxStyle("Round", pad=0.2),
                color = '#FFD700',
                zorder = 5
            )
            ax.add_artist(box)
            text = ax.text(
                -0.8, 0.1+g,
                f'Girl {g}', 
                color='w', 
                style = 'italic',
                weight='bold', 
                fontsize=7, 
                ha='center', 
                va='center',
                zorder = 10)
            
            girls_box[g] = {'box':box,'text':text,'x':-0.8,'y':g+0.1}
        boys_box = {}
        c = 0
        for b in boy_set:
            box = patch.FancyBboxPatch(
                (1, c), 0.4, 0.2,
                boxstyle=patch.BoxStyle("Round", pad=0.2),
                color = '#0066CC',
                zorder = 5
            )
            ax.add_artist(box)
            text = ax.text(
                1.2, 0.1+c,
                f'Boy {b}', 
                color='w', 
                style = 'italic',
                weight='bold', 
                fontsize=7, 
                ha='center', 
                va='center',
                zorder = 10
            )
            boys_box[b] = {'box':box,'text':text,'x':1.2,'y':c+0.1}
            c += 1
        hint_text.set_text(f'Try to match {len(girls)} girls')
        if time <= real_time:
            time += 1
        else:
            real_time += 1
            camera.snap()
            continue
        
        girls_matched = {}
        boys_matched = {}
        for g0 in range(len(girls)):
            girls_visited = {}
            boys_visited = {}
            stack = [g0]
            flag = True
            while stack and flag:
                g = stack.pop(0)    
                girls_box[g]['box'].set_color('k')
                hint_text.set_text(f'Check all the boys girl {g} knows')
                if time <= real_time:
                    time += 1
                else:
                    camera.snap()
                    real_time += 1
                    escape = True
                    break
                
                for b in girls[g]:
                    if b not in boys_visited:
                        if b not in boys_matched:
                            hint_text.set_text(f'Boy {b} has NOT been matched')
                            flag = False
                            boys_matched[b] = g
                            girls_matched[g] = b
                            lines[(g,b)].set_color('#FF0000')
                            while g in girls_visited:
                                b = girls_visited[g]
                                lines[(g,b)].set_color('#808080')
                                g = boys_visited[b]
                                boys_matched[b] = g
                                girls_matched[g] = b
                                lines[(g,b)].set_color('#FF0000')
                            break
                        else:
                            boys_box[b]['box'].set_color('k')
                            hint_text.set_text(f'Boy {b} has been matched')
                            if time <= real_time:
                                time += 1
                            else:
                                real_time += 1
                                escape = True
                            
                                camera.snap()
                                break
                            
                            newg = boys_matched[b]
                            stack.append(newg)
                            boys_visited[b] = g
                            girls_visited[newg] = b
                            
                            if time <= real_time:
                                time += 1
                            else:
                                real_time += 1
                                escape = True
                            
                                camera.snap()
                                break
                if escape:
                    break
            if escape:
                break

            if not flag:
                hint_text.set_text(f'Expand matching!')
            else:
                hint_text.set_text(f'No unmatched boy is found!')
                
            for b in boys_visited:
                boys_box[b]['box'].set_color('#0066CC')
            for g in girls_visited:
                girls_box[g]['box'].set_color('#FFD700')
            girls_box[g0]['box'].set_color('#FFD700')
            if time <= real_time:
                time += 1
            else:
                real_time += 1
                escape = True          
                camera.snap()
                break
        if escape:
            continue

        if len(girls_matched) == len(girls):
            hint_text.set_text(f"Hall's marriage condition is satisfies!")
        else:
            hint_text.set_text(f"Hall's marriage condition is NOT satisfies!")

                            
        camera.snap()
        break
    with output_widget:
        clear_output()
        ani = camera.animate(interval = 1000)
        display(ani)


input_text.observe(HKK, names = 'value')
display(output_widget)

Text(value='', continuous_update=False, description='Input array', placeholder='Input a 2D list like [[0,],[2,…

Output()